In [1]:
import os
import openai
import sys
import langchain

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

openai.api_key  = os.environ['OPENAI_API_KEY']

langchain.debug=True

## Scrabing Pages

### Loading URLS

In [7]:
import pickle

# Load dictionary from the file
with open('data/erasmus_page_urls.pickle', 'rb') as handle:
    url_dict = pickle.load(handle)

url_dict

{'http://erasmus.yildiz.edu.tr': True,
 'https://erasmus.yildiz.edu.tr/': True,
 'https://erasmus.yildiz.edu.tr/sayfa/AB/Hakkımızda/625': True,
 'https://erasmus.yildiz.edu.tr/sayfa/6': True,
 'https://erasmus.yildiz.edu.tr/mansetler/10/Erasmus-Uygulaması-ve-Avrupa-Öğrenci-Kartı-Girişimi': True,
 'https://erasmus.yildiz.edu.tr/mansetler/3/Erasmus-': True,
 'https://erasmus.yildiz.edu.tr/sayfa/7/2024-2025-Başvuruları-için-Süreç-Takvimi/911': True,
 'https://erasmus.yildiz.edu.tr/sayfa/7/Erasmus-Süreç-Takvimi-2023-2024/810': True,
 'https://erasmus.yildiz.edu.tr/sayfa/7/Bölüm-Koordinatörleri-Departmental-Coordinators/737': True,
 'https://erasmus.yildiz.edu.tr/media/files/Fact Sheet HKA_2023(1).pdf': 'PDF',
 'https://erasmus.yildiz.edu.tr/media/files/2022_08_01_KIT_D_KARLSRU01_ERASMUS_Factsheet(1).pdf': 'PDF',
 'https://erasmus.yildiz.edu.tr/media/files/Hochschule Mainz(1).pdf': 'PDF',
 'https://erasmus.yildiz.edu.tr/media/files/Fact Sheet for Partners 2023_2024_DMUNSTER02.pdf': 'PDF',
 

### Scraping

In [45]:
common1 = """\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n \n \n \n \n \n

 Anasayfa
 HAKKIMIZDA
 İLETİŞİM
 ENGLISH


\n\n\n\n\n\n\n\n\n\n



YILDIZ TEKNİK ÜNİVERSİTESİ
ERASMUS + PROGRAM BİRİMİ


\n\n\n
"""

common2 = """

HIZLI ERİŞİM


2024-2025 Başvuruları için Süreç Takvimi


Erasmus Süreç Takvimi 2023-2024


Bölüm Koordinatörleri-Departmental Coordinators


Fact Sheet of Host Universities


Bölüm Alan Kodları-ISCED Codes


OLS Çevrimiçi Dil Desteği-Online Linguistic Support


Ek Hibe Destekleri (Dezavantajlı Geçmiş-Yeşil Seyahat)-Additional Grant Support


Vize Duyuruları


SSS-FAQ


"""

In [46]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.document_loaders import WebBaseLoader


def process_urls(url_dict):
    web_docs = []
    unprocessed_urls = []

    for i, url in enumerate(url_dict):
        try:
            print(f'Processing url {i+1}')
            if url_dict[url] != 'PDF':
                web_loader = WebBaseLoader(url)
                web_doc = web_loader.load()
                edited_content = web_doc[0].page_content[:-95].replace(common1, "").replace(common2, "")
                while "\n\n" in edited_content:
                    edited_content = edited_content.replace("\n\n", "")
                web_doc[0].page_content = edited_content
                web_docs.append(web_doc[0])
            else:
                loader = PyPDFLoader(url)
                pages = loader.load()
                edited_content = pages[0].page_content
                while "\n\n\n" in edited_content:
                    edited_content = edited_content.replace("\n\n\n", "")
                web_docs.append(pages[0])
        except Exception as e:
            print(f'Error processing url {i+1}: {str(e)}')
            unprocessed_urls.append(url)

    print('Mission Completed')
    return web_docs, unprocessed_urls

web_docs, unprocessed_urls = process_urls(url_dict)

Processing url 1
Processing url 2
Processing url 3
Processing url 4
Processing url 5
Processing url 6
Processing url 7
Processing url 8
Processing url 9
Processing url 10
Processing url 11
Processing url 12
Processing url 13
Processing url 14
Processing url 15
Processing url 16
Processing url 17
Processing url 18
Processing url 19
Processing url 20
Processing url 21
Processing url 22
Processing url 23
Processing url 24
Processing url 25
Processing url 26
Processing url 27
Processing url 28
Processing url 29
Processing url 30
Processing url 31
Processing url 32
Processing url 33
Processing url 34
Processing url 35
Processing url 36
Processing url 37
Processing url 38
Processing url 39
Processing url 40
Processing url 41
Processing url 42
Processing url 43
Processing url 44
Processing url 45
Processing url 46
Processing url 47
Processing url 48
Processing url 49
Processing url 50
Processing url 51
Processing url 52
Processing url 53
Processing url 54
Processing url 55
Processing url 56
P

### Text Splitting

In [125]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

r_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1024, 
    chunk_overlap=256,
    separators=["\n\n", "\n", ". ", " ", ""]
)

splitted_docs = r_splitter.split_documents(web_docs)
splitted_docs = [split for split in splitted_docs if len(split.page_content) > 150]
len(splitted_docs)

810

### Adding Metadata to the documents for better Retrieval

In [126]:
from urllib.parse import urljoin, urlparse
from tqdm import tqdm

def extract_text_from_url(url):
    """
    Extract text from the end of the URL, correctly handling /text and /text/number cases.
    """
    # Extracting the path from the URL and splitting it
    path_segments = urlparse(url).path.split('/')
    # Filtering out empty segments and digits-only segments
    text_segments = [segment for segment in path_segments if segment and not segment.isdigit()]

    # The last segment in text_segments should be the text we want to extract
    if text_segments:
        return text_segments[-1]  # Return the last text part
    return None


def process_documents(splitted_docs):
    """
    Adds the metadata at the beginning of each document
    """
    for doc in tqdm(splitted_docs, desc="Processing documents"):
        if not doc.metadata['source'].endswith('.pdf'):
            doc.page_content = f"Source: {doc.metadata['source']}\nPage Title: {doc.metadata['description']}\nSubject: {extract_text_from_url(doc.metadata['source'])}\n\n" + doc.page_content
        else:
            doc.page_content = f"Source: {doc.metadata['source']}\nSubject: {extract_text_from_url(doc.metadata['source'])}\nPdf Page Number: {doc.metadata['page']}\n\n" + doc.page_content

            
process_documents(splitted_docs)

Processing documents: 100%|████████████████| 810/810 [00:00<00:00, 73999.40it/s]


In [127]:
print(splitted_docs[20].page_content)
print('\n---------------------\n')
print(splitted_docs[40].page_content)

Source: https://erasmus.yildiz.edu.tr/sayfa/7/2024-2025-Başvuruları-için-Süreç-Takvimi/911
Page Title:  Erasmus + Program Birimi Resmi Sitesi
Subject: 2024-2025-Başvuruları-için-Süreç-Takvimi

Kabul mektubu kriterleri için: erasmus.yildiz.edu.tr adresini ziyaret ediniz.2. Periyod 01 Ocak 2025 - 30 Eylül 2025 tarihleri arasında gerçekleştirilmelidir. 
Öğrenim Hareketliliği Yerleştirme listelerinin ilanı29 Mart 2024Erasmus+ Program Birimi, başarı puanlarına göre anlaşmalı üniversitelere yerleştirilen adayların listesini web sayfasında ilan edecektir. Staj hareketliliğinde listeler hibeler ile birlikte açıklanacaktır.Değişiklik ve feragat taleplerinin alınması için son tarihler 15 Eylül 2024 (tüm dönemler için)                      15 Kasım 2024 (sadece bahar dönemi için)                             Öğrenciler belirtilen tarihler arasında değişiklik ve feragat taleplerini Erasmus+ Programı Birimine iletebilirler. Her öğrenci sadece bir defa değişiklik talebinde bulunabilir. Değişiklik tal

## Enhancing Metadata

In [128]:
i = 0
for doc in splitted_docs:
    if 'haberler' in doc.metadata['source']:
        i +=1
        print(doc.metadata['source'])
        if i > 4:
            break

https://erasmus.yildiz.edu.tr/haberler
https://erasmus.yildiz.edu.tr/haberler
https://erasmus.yildiz.edu.tr/haberler/634/2024-2025-AKADEMİK-YILI-ERASMUS--PROGRAMI-YABANCI-DİL-YETERLİK-SINAVI-ve-SINAVA-GİRECEK-ÖĞRENCİLERİN-LİSTESİ-HAKKINDA
https://erasmus.yildiz.edu.tr/haberler/634/2024-2025-AKADEMİK-YILI-ERASMUS--PROGRAMI-YABANCI-DİL-YETERLİK-SINAVI-ve-SINAVA-GİRECEK-ÖĞRENCİLERİN-LİSTESİ-HAKKINDA
https://erasmus.yildiz.edu.tr/haberler/634/2024-2025-AKADEMİK-YILI-ERASMUS--PROGRAMI-YABANCI-DİL-YETERLİK-SINAVI-ve-SINAVA-GİRECEK-ÖĞRENCİLERİN-LİSTESİ-HAKKINDA


In [133]:
def enhance_meta_data(docs):
    '''
    Check if the document is an anouncement/duyuru/haber document and adds
    this information to the metada. 
    '''
    for doc in tqdm(docs, desc="Processing documents"):
        if 'haberler' in doc.metadata['source']:
            doc.metadata['Duyuru'] = 1
        else:
            doc.metadata['Duyuru'] = 0
            
enhance_meta_data(splitted_docs)

Processing documents: 100%|███████████████| 810/810 [00:00<00:00, 701359.67it/s]


In [134]:
splitted_docs[15]

Document(page_content="Source: https://erasmus.yildiz.edu.tr/sayfa/7/2024-2025-Başvuruları-için-Süreç-Takvimi/911\nPage Title:  Erasmus + Program Birimi Resmi Sitesi\nSubject: 2024-2025-Başvuruları-için-Süreç-Takvimi\n\n. Başvuru çağrı metninde belirtilen asgari şartları sağlayamayan öğrencilerin bu başvuru için süreçleri noktalanmış olup Yabancı Dil Sınavına giremeyeceklerdir. Ders aldığınız Öğretim Üyelerinin notlarınızı eksiksiz girdiğinden emin olunuz. Bu tarihten sonra güncellenen AGNO'lar dikkate alınmayacaktır.Yabancı dil sınavına girmeye hak kazanan öğrencilerin ilanı19 Şubat 2024Ön başvurusu kabul edilen, asgari şartları sağlayan adaylardan Yabancı Dil Sınavına girecek olanlar için sınav yer ve zaman bilgileri Erasmus+ Program Birimi web sayfasında duyurulacaktır, kişiye özel bilgilendirme yapılmayacaktır.Yabancı dil sınavı2 Mart 2024Sınav fiziki olarak yapılacaktır", metadata={'source': 'https://erasmus.yildiz.edu.tr/sayfa/7/2024-2025-Başvuruları-için-Süreç-Takvimi/911', 'tit

## Vectorstores

In [135]:
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma

embedding = OpenAIEmbeddings()

erasmus_vectordb = Chroma.from_documents(
    documents=splitted_docs,
    embedding=embedding,
    persist_directory='./db/erasmus_website'
)
erasmus_vectordb.persist()

# erasmus_vectordb = Chroma(persist_directory='./db/erasmus_website', embedding_function=embedding)

In [136]:
erasmus_vectordb.persist()
